In [1]:
import cv2
import numpy as np
from os.path import isfile, join
import h5py
from skimage.util import view_as_blocks
from sklearn.cluster import KMeans

In [2]:
def hist_match(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

def hist_match_color(source,template):
    b_source=source[:,:,0]
    g_source=source[:,:,1]
    r_source=source[:,:,2]
    
    b_template=template[:,:,0]
    g_template=template[:,:,1]
    r_template=template[:,:,2]
    
    b_matched=hist_match(b_source,b_template)
    g_matched=hist_match(g_source,g_template)
    r_matched=hist_match(r_source,r_template)
    
    matched=cv2.merge([b_matched,g_matched,r_matched])
    return matched

In [3]:
filename='/home/xli/jp2_Temp/PMD1464&1463-F31-2014.02.01-04.12.52_PMD1463_1_0091_lossless.jp2'
img = cv2.imread(filename,-1)

In [4]:
#use k mean with many clusters to classify, and extract layer informations
#test if this can tell the autofluo and red tracer
img_source = np.copy(img)
img8 = np.asarray(img/float(16),'uint8')
template = cv2.imread('/home/xli/jp2_Temp/Thalamus/Thalamus_0.jpg')

img8 = hist_match_color(img8, template)
cv2.imwrite('/home/xli/jp2_Temp/0091.jp2', img8)

True

In [5]:
data=img.reshape([img.shape[0]*img.shape[1], 3])

In [ ]:
y_predicted=KMeans(n_clusters = 12,max_iter=200,tol=0.01).fit_predict(data)